DATA MINING SUPERMARKET_SALES

1. Liệu giới tính có liên quan đến loại sản phẩm được mua ? Dự đoán loại sản phẩm mà người phụ nữ tiếp theo sẽ mua.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression


In [4]:
df1 = pd.read_excel("supermarket_mining.xlsx", sheet_name= "Câu 1")
df1 = df1.drop('InvoiceID_SK.Invoice ID SK', axis=1)
mapping = {'Gender.Gender': 'Gender','Product Line.Product Line': 'Product Line',
           'Customer Type.Customer Type': 'Customer Type',
           'Payment.Payment Type':'Payment'}
df1 = df1.rename(columns= mapping)
df1.head()


,Gender,Product Line,Quantity,Customer Type,Payment,Total
0,Female,Health and beauty,7,Member,Ewallet,496.6885
1,Female,Home and lifestyle,6,Member,Ewallet,392.3880
2,Female,Health and beauty,2,Member,Credit card,68.8940
3,Female,Sports and travel,7,Member,Credit card,458.3845
4,Female,Electronic accessories,8,Member,Credit card,399.6840


In [12]:
print("Sử dụng Chi-square để xem giới tính và sản phẩm được mua có liên quan đến nhau hay không")
print("H0: Không có sự liên quan đáng kể giữa giới tính và loại sản phẩm được mua ")
print("H1: Có ít nhất một sự khác bị đáng kể giữa giới tính và loại sản phẩm được mua")
contingency_gender = pd.crosstab(df1['Gender'], df1['Product Line'])

chi2, p, _, _ = chi2_contingency(contingency_gender)

print(f'Chi-square statistic: {chi2:.4f}')
print(f'P-value: {p:.4f}')

alpha = 0.05
if p < alpha:
    print("Có đủ bằng chứng để bác bỏ giả thiết H0: Giới tính ảnh hưởng đến loại sản phẩm được mua.")
    print("Như vậy giới tính không có ảnh hưởng đến loại sản phẩm được mua")
else:
    print("Không đủ bằng chứng để bác bỏ giả thiết H0: Không có sự ảnh hưởng đáng kể của giới tính đối với loại sản phẩm được mua.")
    print("Giới tính và loại sản phẩm được mua không có sự liên quan đáng kể đến nhau")

Sử dụng Chi-square để xem giới tính và sản phẩm được mua có liên quan đến nhau hay không
H0: Không có sự liên quan đáng kể giữa giới tính và loại sản phẩm được mua 
H1: Có ít nhất một sự khác bị đáng kể giữa giới tính và loại sản phẩm được mua
Chi-square statistic: 3.3364
P-value: 0.6483
Không đủ bằng chứng để bác bỏ giả thiết H0: Không có sự ảnh hưởng đáng kể của giới tính đối với loại sản phẩm được mua.
Giới tính và loại sản phẩm được mua không có sự liên quan đáng kể đến nhau


2. Cho một khách hàng có giới tính là Female, hiện đang là Member của siêu thị, và cô ấy có thói quen sử dụng Creadit card, dự đoán lần tiếp theo cô ấy mua hàng thì tổng giá trị đơn hàng là bao nhiêu

In [6]:

features = ['Gender', 'Customer Type', 'Payment']
target = 'Total'


train_data, test_data = train_test_split(df1, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), features)
    ])

model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


model.fit(train_data[features], train_data[target])


predictions = model.predict(test_data[features])


mse = mean_squared_error(test_data[target], predictions)
print(f'Độ lỗi: {mse}')

new_data = pd.DataFrame({'Gender': ['Female'], 'Customer Type': ['Member'], 'Payment': ['Credit card']})
prediction_new_data = model.predict(new_data)

print(f'Dự đoán cô ấy sẽ mua hàng có tổng giá tiền là: {prediction_new_data[0]}')

print('\nỞ mô hình hiện tại có độ lỗi rất cao, thế nên đây không phải là mô hình hoàn hảo, sẽ có sự chênh lệch lớn giữa giá trị dự đoán và giá trị thực tế')
print('Tuy nhiên, với dataset hiện tại ta không tìm được mô hình phù hợp hơn')

Độ lỗi: 43781.107192545
Dự đoán cô ấy sẽ mua hàng có tổng giá tiền là: 288.0

Ở mô hình hiện tại có độ lỗi rất cao, thế nên đây không phải là mô hình hoàn hảo, sẽ có sự chênh lệch lớn giữa giá trị dự đoán và giá trị thực tế
Tuy nhiên, với dataset hiện tại ta không tìm được mô hình phù hợp hơn


3. Siêu thị chuẩn bị mở 1 chi nhánh D mới ở thành phố HCM, dự đoán doanh thu chi nhánh mới với dữ liệu support cho sẵn

In [7]:
df2 = pd.read_excel("supermarket_mining.xlsx", sheet_name= "Câu 2")
mapping = {'City.Branch': 'Branch','City.City': 'City','Gender.Gender': 'Gender'}
df2 = df2.rename(columns= mapping)
df2.head()




,Branch,City,Gross Income
0,A,Yangon,466498.540486
1,B,Mandalay,456669.704771
2,C,Naypyitaw,459304.461914


In [8]:
df2_sp = pd.read_excel("supermarket_mining.xlsx", sheet_name= "Câu 2 support")
df2_sp = df2_sp.rename(columns= {'City.City':'City'})
df2_sp.head()

,City,Population,Female,Male,Rival
0,HCM,7200,49,51,9
1,Mandalay,5000,30,70,5
2,Naypyitaw,6000,42,48,9
3,Yangon,7300,59,41,2


In [9]:
df_combined = pd.merge(df2, df2_sp, on='City')
df_combined.head()

,Branch,City,Gross Income,Population,Female,Male,Rival
0,A,Yangon,466498.540486,7300,59,41,2
1,B,Mandalay,456669.704771,5000,30,70,5
2,C,Naypyitaw,459304.461914,6000,42,48,9


In [10]:

features = ['Branch', 'City', 'Population', 'Female', 'Male', 'Rival']
target = 'Gross Income'

train_data, test_data = train_test_split(df_combined, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Branch', 'City'])
    ])

model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model.fit(train_data[features], train_data[target])


predictions = model.predict(test_data[features])


mse = mean_squared_error(test_data[target], predictions)
print(f'Độ lỗi: {mse}')

HCM_DATA = df2_sp[df2_sp['City'] =='HCM']
new_branch_data = pd.DataFrame({'Branch': ['D'], 'City': ['HCM'], 'Population': [HCM_DATA['Population']], 'Female': [HCM_DATA['Female']], 'Male': [HCM_DATA['Male']], 'Rival': [HCM_DATA['Rival']]})
prediction_new_branch = model.predict(new_branch_data[features])

print(f'Dự đoán doanh thu của chi nhánh mới: {prediction_new_branch[0]}')
print('\n Tuy độ lỗi quá cao nhưng tạm chấp nhận vì dữ liệu không đầy đủ')

Độ lỗi: 72444902.69759203
Dự đoán doanh thu của chi nhánh mới: 457987.0833424933

 Tuy độ lỗi quá cao nhưng tạm chấp nhận vì dữ liệu không đầy đủ
